In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import numpy as np
from cipher import Ring, Parameters, Context 
from scheme import Evaluator, Encryptor, Decryptor

## Context set up

In [28]:
myring = Ring(seed=1234)
parms = Parameters(logp = 30, logq = 150, logn = 12)
context = Context(parms, myring)

sk = context.generate_secret_key()

print(context.params)

 logp: 30
 logq: 150 
 logn: 12


## Create Agents

In [32]:
keys = {"mult":context.generate_mult_key(),
        "rot":{'1':'hi1',
               '2':'hi2',
               '4':'hi4',
               '8':'hi8'}}

evalu = Evaluator(keys)
encryptor = Encryptor(context)

decryptor = Decryptor(sk)

## Create a ctxt and attempt to print it 
Both ndarray and list are OK

In [34]:
ctxt1 = encryptor.encrypt(np.array([1,2,3,4]))
ctxt2 = encryptor.encrypt([2,3,3,5])
print(ctxt1)

You can't read the content


#### You need to first decrypt it

In [35]:
decrypted = decryptor.decrypt(ctxt2)
print(decrypted)

[2. 3. 3. ... 0. 0. 0.]


## Addition

In [37]:
new_ctxt = evalu.add(ctxt1, ctxt2, inplace=False)

print(decryptor.decrypt(new_ctxt))

[3. 5. 6. ... 0. 0. 0.]


### Mult

In [38]:
ctxt3 = evalu.mult(ctxt1, ctxt2, inplace=False)

print(decryptor.decrypt(ctxt3))

evalu.mult(ctxt1, ctxt2, inplace=True)
print(decryptor.decrypt(ctxt1))

[2. 6. 9. ... 0. 0. 0.]
[2. 6. 9. ... 0. 0. 0.]
